# TCN

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import sys
sys.path.append("/home/angrypark/korean-text-matching-tf/")

In [10]:
import tensorflow as tf
import numpy as np
import argparse
from datetime import datetime

from data_loader import DataGenerator
from trainer import MatchingModelTrainer
from preprocessor import DynamicPreprocessor
from utils.dirs import create_dirs
from utils.logger import SummaryWriter
from utils.config import load_config, save_config
from models.base import get_model
from utils.utils import JamoProcessor

In [11]:
class Config:
    def __init__(self):
        self.mode = "train"
        self.name = "debug_embedding"
        self.config = ""
        self.train_dir = "/media/scatter/scatterdisk/reply_matching_model/debug/"
        self.val_dir = "/media/scatter/scatterdisk/reply_matching_model/debug/sol.small.txt"
        self.pretrained_embed_dir = "/media/scatter/scatterdisk/pretrained_embedding/fasttext.sent_piece_50K.256D"
        self.checkpoint_dir = "/home/angrypark/"
        self.model = "DualEncoderLSTM"
        self.sent_piece_model = "/media/scatter/scatterdisk/tokenizer/sent_piece.50K.model"
        self.soynlp_scores = "/media/scatter/scatterdisk/tokenizer/soynlp_scores.sol.100M.txt"
        self.normalizer = "DummyNormalizer"
        self.tokenizer = "SentencePieceTokenizer"
        self.vocab_size = 50000
        self.vocab_list = "/media/scatter/scatterdisk/pretrained_embedding/vocab_list.sent_piece_50K.txt"
        
        self.embed_dim = 256
        self.learning_rate = 1e-3
        self.min_length = 1
        self.max_length = 20
        self.dropout_keep_prob = 0.9
        
        self.lstm_dim = 512
        self.tcn_num_channels = 3
        self.tcn_kernel_size = 2
        
        self.batch_size = 512
        self.num_epochs = 30
        self.evaluate_every = 50
        self.save_every = 100
        
        self.max_to_keep = 1
        self.shuffle = True
        
        self.negative_sampling = "random"
        self.num_negative_samples = 4

config = Config()

In [12]:
config = create_dirs(config)
device_config = tf.ConfigProto()
device_config.gpu_options.allow_growth = True
sess = tf.Session(config=device_config)
preprocessor = DynamicPreprocessor(config)
data = DataGenerator(preprocessor, config)
summary_writer = SummaryWriter(sess, config)
trainer = MatchingModelTrainer(sess, preprocessor, data, config, summary_writer)

In [13]:
model, sess = trainer.build_graph(name="train")

[11:31:34][INFO] Building train graph... 


Pre-trained embedding loaded. Number of OOV : 4293 / 50000


/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
[11:32:29][INFO] Loading checkpoint from /home/angrypark/debug_embedding/ 
[11:32:29][ERROR] No checkpoint found in /home/angrypark/debug_embedding/ 


In [14]:
batch_queries, batch_replies, \
batch_queries_lengths, batch_replies_lengths = next(trainer.train_iterator)

cur_batch_length = len(batch_queries)

feed_dict = {model.input_queries: batch_queries,
             model.input_replies: batch_replies,
             model.queries_lengths: batch_queries_lengths,
             model.replies_lengths: batch_replies_lengths,
             model.dropout_keep_prob: 1
            }

In [15]:
embedding = model.embedding_lookup.eval(feed_dict=feed_dict, session=sess)

In [17]:
embedding.shape

(512, 20, 256)

In [8]:
from models.dual_encoder_lstm import get_embeddings
with tf.Graph().as_default():
    embeddings = get_embeddings(preprocessor.vectorizer.idx2word, config)

Pre-trained embedding loaded. Number of OOV : 4293 / 50000


In [10]:
embedding_lookup = model.embedding_look

(50000, 256)

In [5]:
trainer.train()

[21:39:33][INFO] Building train graph... 


Pre-trained embedding loaded. Number of OOV : 4293 / 50000


/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
[21:39:46][INFO] Loading checkpoint from /home/angrypark/debug_tcn/ 
[21:39:46][ERROR] No checkpoint found in /home/angrypark/debug_tcn/ 
[21:39:46][INFO] Building val graph... 


Pre-trained embedding loaded. Number of OOV : 4293 / 50000


  0%|          | 48/583116 [00:13<47:04:44,  3.44it/s][21:40:09][ERROR] No checkpoint found in /home/angrypark/debug_tcn/ 

  0%|          | 98/583116 [00:25<42:49:23,  3.78it/s]

...

INFO:tensorflow:Restoring parameters from /home/angrypark/debug_tcn/model.ckpt



100%|██████████| 2/2 [00:00<00:00,  6.14it/s]
[21:40:54][WARNING] [Step 200] Saving for best loss : 4494.59180 -> 3015.27734 
[21:40:55][WARNING] Epoch :  1 | Step :      199 | Train loss : 3375.7354 | Train accuracy : 0.6877 | Val loss : 3015.2773 | Val accuracy : 0.6820  
  0%|          | 248/583116 [01:10<45:57:43,  3.52it/s]

INFO:tensorflow:Restoring parameters from /home/angrypark/debug_tcn/model.ckpt



100%|██████████| 2/2 [00:00<00:00,  3.60it/s]
[21:41:06][WARNING] [Step 250] Saving for best loss : 3015.27734 -> 3003.04761 
[21:41:08][WARNING] Epoch :  1 | Step :      249 | Train loss : 2871.1868 | Train accuracy : 0.6941 | Val loss : 3003.0476 | Val accuracy : 0.6743  
  0%|          | 269/583116 [01:17<46:27:33,  3.48it/s]

KeyboardInterrupt: 

In [1]:
0.96**8

0.7213895789838334

In [2]:
1e-3*(0.96**8)

0.0007213895789838334

In [3]:
3e-4*(0.96**8)

0.00021641687369514998

In [3]:
from tqdm import tqdm

In [4]:
loop = tqdm(range(1,100))

  0%|          | 0/99 [00:00<?, ?it/s]

In [5]:
for i in range(10):
    for step in loop:
        pass

100%|██████████| 99/99 [00:20<00:00,  4.80it/s]
